# final code at the end

In [ ]:
%%capture
!pip install langchain-community langchain_groq chromadb gradio
!pip install langchain-google-genai google-generativeai

In [ ]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq

In [ ]:
anime= pd.read_csv("/content/drive/MyDrive/anime_with_synopsis.csv")
anime

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
264,296,Dragon Drive,6.73,"Action, Sci-Fi, Adventure, Comedy, Fantasy, Sc...","If there's one word to describe Reiji Ozora, i..."
265,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",ushuna is a blonde and very beautiful Senshi (...
266,298,.hack//Tasogare no Udewa Densetsu,6.60,"Adventure, Comedy, Fantasy, Game, Sci-Fi, Shounen","g the legendary characters ""Kite"" and ""Black R..."
267,299,.hack//Liminality,6.61,"Game, Mystery, Sci-Fi","hile playing the newly released MMORPG ""The Wo..."


In [ ]:
anime.dropna()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
264,296,Dragon Drive,6.73,"Action, Sci-Fi, Adventure, Comedy, Fantasy, Sc...","If there's one word to describe Reiji Ozora, i..."
265,297,Grenadier: Hohoemi no Senshi,6.72,"Action, Ecchi, Adventure, Comedy, Shounen",ushuna is a blonde and very beautiful Senshi (...
266,298,.hack//Tasogare no Udewa Densetsu,6.60,"Adventure, Comedy, Fantasy, Game, Sci-Fi, Shounen","g the legendary characters ""Kite"" and ""Black R..."
267,299,.hack//Liminality,6.61,"Game, Mystery, Sci-Fi","hile playing the newly released MMORPG ""The Wo..."


In [ ]:
anime['combined_df'] =anime.apply(lambda row: f"Title: {row['Name']}. Overview : {row['sypnopsis']} Genres : {row['Genres']}", axis = 1)

In [ ]:
anime['combined_df'][0]

'Title: Cowboy Bebop. Overview : In the year 2071, humanity has colonized several of the planets and moons of the solar system leaving the now uninhabitable surface of planet Earth behind. The Inter Solar System Police attempts to keep peace in the galaxy, aided in part by outlaw bounty hunters, referred to as "Cowboys." The ragtag team aboard the spaceship Bebop are two such individuals. Mellow and carefree Spike Spiegel is balanced by his boisterous, pragmatic partner Jet Black as the pair makes a living chasing bounties and collecting rewards. Thrown off course by the addition of new members that they meet in their travels—Ein, a genetically engineered, highly intelligent Welsh Corgi; femme fatale Faye Valentine, an enigmatic trickster with memory loss; and the strange computer whiz kid Edward Wong—the crew embarks on thrilling adventures that unravel each member\'s dark and mysterious past little by little. Well-balanced with high density action and light-hearted comedy, Cowboy Beb

In [ ]:
anime[['combined_df']].to_csv('anime_updated.csv', index= False)

In [ ]:
pd.read_csv("/content/anime_updated.csv")

,combined_df
0,Title: Cowboy Bebop. Overview : In the year 20...
1,Title: Cowboy Bebop: Tengoku no Tobira. Overvi...
2,Title: Trigun. Overview : Vash the Stampede is...
3,Title: Witch Hunter Robin. Overview : ches are...
4,Title: Bouken Ou Beet. Overview : It is the da...
...,...
264,Title: Dragon Drive. Overview : If there's one...
265,Title: Grenadier: Hohoemi no Senshi. Overview ...
266,Title: .hack//Tasogare no Udewa Densetsu. Over...
267,Title: .hack//Liminality. Overview : hile play...


In [ ]:
loader = CSVLoader(file_path="/content/anime_updated.csv")
data = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_path = "./chroma_db"
docssearch = Chroma.from_documents(texts, embedding=embeddings, persist_directory=db_path)
retriever = docssearch.as_retriever()
llm = ChatGroq(api_key="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", model="llama3-70b-8192", temperature=0)

In [ ]:
query = "I am looking for an animated sci-fi movie. What could you suggest to me"
docs = docssearch.similarity_search(query, k =1)
docs

[Document(metadata={'source': '/content/anime_updated.csv', 'row': 161}, page_content='combined_info: Title: Xenosaga The Animation. Overview: The year is T.C. 4767. Four thousand years have passed since humanity abandoned its birthplace, the planet Earth. Beset by the hostile alien Gnosis, mankind is now scrambling to find ways to defeat this threat to their existence. The development of KOS-MOS (a specialized android with amazing capabilities) by Vector engineer Shion Uzuki was one response to the threat. But when their ship is destroyed by the Gnosis, Shion and her companions find themselves thrust into the middle of a battle with no clear sides... (Source: Anime-Planet) Genres: Action, Mecha, Sci-Fi, Shounen, Space')]

# Using QA Retrieval for movie recommendation

In [ ]:
qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True)

In [ ]:
query = "Can you recommend a romance anime with a unique storyline?"
result = qa({"query": query})

In [ ]:
result['result']

In [ ]:
result['source_documents'][0]

In [ ]:
from langchain.prompts import PromptTemplate

template = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, suggest three anime, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT  =  PromptTemplate(
    template=template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Give me a list of comedy animes that are lighthearted and fun."
result = qa({"query": query})
print(result['result'])

In [ ]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end.
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'male')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

In [ ]:
PROMPT  =  PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm,
            chain_type = "stuff",
            retriever = docssearch.as_retriever(),
            return_source_documents = True,
                                 chain_type_kwargs = chain_type_kwargs)

query = "Which anime would you suggest for someone new to anime?"
result = qa({"query": query})
print(result['result'])

In [ ]:
import os
import gradio as gr
import pandas as pd
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter

csv_file_path = "/content/drive/MyDrive/anime_with_synopsis.csv"
updated_csv_path = "/content/anime_updated.csv"

anime = pd.read_csv(csv_file_path)
anime = anime.dropna()
anime['combined_info'] = anime.apply(
    lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']}. Genres: {row['Genres']}", axis=1
)
anime[['combined_info']].to_csv(updated_csv_path, index=False)
db_path = "./chroma_db"
os.makedirs(db_path, exist_ok=True)

loader = CSVLoader(file_path=updated_csv_path)
data = loader.load()

if not data:
    print("Error: No data found in the dataset. Ensure the CSV file has valid entries.")

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docssearch = Chroma.from_documents(texts, embedding=embeddings, persist_directory=db_path)
retriever = docssearch.as_retriever()

llm = ChatGroq(
    api_key="Your own API",
    model="llama3-70b-8192",
    temperature=0
)

template = """You are an anime recommender system that helps users find anime that match their preferences.
Use the following context to answer the question at the end.
Suggest three anime with a short description and why the user might like them.
If you don't know the answer, return "Not sufficient data".

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    try:
        response = qa_chain({"query": query})
        return response.get("result", "Not sufficient data")
    except Exception as e:
        return f"Error: {str(e)}"

theme = "Respair/Shiki@1.2.1"
iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme=theme,
)
docssearch.persist()
if __name__ == "__main__":
    iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d7e00c86b32acb549.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# with gemini


In [ ]:
import os
import gradio as gr
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI

csv_file_path = "/content/drive/MyDrive/anime_with_synopsis.csv"
updated_csv_path = "/content/drive/MyDrive/anime_updated.csv"

if os.path.exists(csv_file_path):
    anime = pd.read_csv(csv_file_path).dropna()
    anime['combined_info'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)
    anime[['combined_info']].to_csv(updated_csv_path, index=False)
else:
    print("Error: anime_with_synopsis.csv not found.")

db_path = "./chroma_db"
os.makedirs(db_path, exist_ok=True)

if os.path.exists(updated_csv_path):
    loader = CSVLoader(file_path=updated_csv_path)
    data = loader.load()
else:
    print("Error: anime_updated.csv not found.")
    data = []

if not data:
    print("Error: No data found in the dataset. Ensure the CSV file has valid entries.")

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
docssearch = Chroma.from_documents(texts, embedding=embeddings, persist_directory=db_path)
retriever = docssearch.as_retriever()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key="Your own API",
    temperature=0
)

template = """You are an anime recommender system that helps users find anime that match their preferences.
Use the following context to answer the question at the end.
Suggest three anime with a short description and why the user might like them.
If you don't know the answer, return "Not sufficient data".

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    try:
        response = qa_chain.invoke({"query": query})
        return response["result"] if "result" in response else "Not sufficient data"
    except Exception as e:
        return f"Error: {str(e)}"

theme = "Respair/Shiki@1.2.1"
iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme=theme,
)

docssearch.persist()

if __name__ == "__main__":
    iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f03a781362b73d4226.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
